In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image

In [64]:
#Data Preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081))
])

In [65]:
#loading mnist dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data', train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.MNIST(
    root='./data', train=False, download=True, transform=transform
)

In [66]:
#dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [67]:
class MNISTClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(784,128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self, x):
        x = self.flatten(x)
        x = self.layers(x)
        return x

In [68]:
#check gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device}')

Using cpu


In [69]:
#intialize model
model = MNISTClassifier() 

In [70]:
#loss function, optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [71]:
#training
def train_epoch(model, train_loader, loss_function, optimizer, device):
    model.train() #puts model in training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        #track progress
        running_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        #print every 100 batches
        if batch_idx % 100 == 0 and batch_idx > 0:
            avg_loss = running_loss / 100
            accuracy = 100. * correct / total
            print(f' [{batch_idx * 64}/{60000}] '
                  f' Loss: {avg_loss:.3f} | Accuracy: {accuracy:.1f}%')
            running_loss = 0.0

In [72]:
#evaluation
def evaluate(model, test_loader, device):
    model.eval() #set to evaluation mode 
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return 100. * correct / total

In [73]:
#training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f'\nEpoch: {epoch+1}')
    train_epoch(model, train_loader, loss_function, optimizer, device)
    accuracy = evaluate(model, test_loader, device)
    print(f'Test Accuracy: {accuracy:.2f}%')


Epoch: 1
 [6400/60000]  Loss: 0.652 | Accuracy: 81.9%
 [12800/60000]  Loss: 0.323 | Accuracy: 86.0%
 [19200/60000]  Loss: 0.289 | Accuracy: 88.0%
 [25600/60000]  Loss: 0.245 | Accuracy: 89.2%
 [32000/60000]  Loss: 0.222 | Accuracy: 90.0%
 [38400/60000]  Loss: 0.209 | Accuracy: 90.7%
 [44800/60000]  Loss: 0.169 | Accuracy: 91.3%
 [51200/60000]  Loss: 0.184 | Accuracy: 91.7%
 [57600/60000]  Loss: 0.161 | Accuracy: 92.1%
Test Accuracy: 95.87%

Epoch: 2
 [6400/60000]  Loss: 0.127 | Accuracy: 96.4%
 [12800/60000]  Loss: 0.126 | Accuracy: 96.2%
 [19200/60000]  Loss: 0.122 | Accuracy: 96.3%
 [25600/60000]  Loss: 0.125 | Accuracy: 96.3%
 [32000/60000]  Loss: 0.113 | Accuracy: 96.3%
 [38400/60000]  Loss: 0.116 | Accuracy: 96.4%
 [44800/60000]  Loss: 0.112 | Accuracy: 96.4%
 [51200/60000]  Loss: 0.117 | Accuracy: 96.4%
 [57600/60000]  Loss: 0.112 | Accuracy: 96.4%
Test Accuracy: 96.84%

Epoch: 3
 [6400/60000]  Loss: 0.080 | Accuracy: 97.4%
 [12800/60000]  Loss: 0.078 | Accuracy: 97.6%
 [19200/6

In [77]:
#testing on image
def predict_image(model, image_path, device):
    model.eval()

    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((28, 28)),
        transforms.functional.invert,
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    img = Image.open(image_path)
    img = transform(img).unsqueeze(0).to(device)  # add batch dimension

    with torch.no_grad():
        output = model(img)
        _, predicted = output.max(1)

    return predicted.item()

In [85]:
digit = predict_image(model, "images.jpg", device)
print("Predicted Digit:", digit)


Predicted Digit: 5
